# Visualizaciones

1. Heatmap: Categorizar en Bins lat/long para crear matriz de 5x160 (10 de largo por region)
    - Temperaturas
    - Precipitaciones
    - Biomas dadas las anteriores [Holdridge](https://en.wikipedia.org/wiki/Holdridge_life_zones)
    - Alturas
    - Equivalencia en regiones
2. Voronoi: Cada estación es un centroide, usar el svg d3 de chile con regiones.

In [1]:
from IPython.display import display
from itertools import product
import pandas as pd
import numpy as np

# Import Data
print('Loading CSVs...', end='')
stations = pd.read_csv(r"cr2_prAmon_2018_stations_ghcn.txt")
station_cols = [
    'codigo_estacion',
    #'institucion',
    #'fuente',
    'nombre',
    'altura',
    'latitud',
    'longitud',
    #'codigo_cuenca',
    #'nombre_cuenca',
    #'codigo_sub_cuenca',
    #'nombre_sub_cuenca',
    #'inicio_observaciones',
    #'fin_observaciones',
    #'cantidad_observaciones',
    #'inicio_automatica',
]

skip = lambda x: x in range(1, 15)

stations = stations[station_cols]
precipit = pd.read_csv(r"cr2_prAmon_2018_ghcn.txt",     skiprows=skip, dtype=str, header=None, low_memory=False).transpose()
tempsavg = pd.read_csv(r"cr2_tasAmon_2018_ghcn.txt",    skiprows=skip, dtype=str, header=None, low_memory=False).transpose()
tempsmin = pd.read_csv(r"cr2_tasminAmon_2018_ghcn.txt", skiprows=skip, dtype=str, header=None, low_memory=False).transpose()
tempsmax = pd.read_csv(r"cr2_tasmaxAmon_2018_ghcn.txt", skiprows=skip, dtype=str, header=None, low_memory=False).transpose()

precipit.rename(columns=precipit.iloc[0], inplace=True)
tempsavg.rename(columns=tempsavg.iloc[0], inplace=True)
tempsmin.rename(columns=tempsmin.iloc[0], inplace=True)
tempsmax.rename(columns=tempsmax.iloc[0], inplace=True)

precipit.drop(precipit.index[0], inplace=True)
tempsavg.drop(tempsavg.index[0], inplace=True)
tempsmin.drop(tempsmin.index[0], inplace=True)
tempsmax.drop(tempsmax.index[0], inplace=True)

print(' DONE')
print('stations:', stations.columns.values)
print('precipit:', precipit.columns.values)
print('tempsavg:', tempsavg.columns.values)
print('tempsmin:', tempsmin.columns.values)
print('tempsmax:', tempsmax.columns.values)

print('\nUn-pivoting and setting types...', end='')

# Un-pivot and set indexes (categorical reduces memory)
precipit = pd.melt(precipit, id_vars=['codigo_estacion'], var_name='date', value_name='precipit')
tempsavg = pd.melt(tempsavg, id_vars=['codigo_estacion'], var_name='date', value_name='tempsavg')
tempsmin = pd.melt(tempsmin, id_vars=['codigo_estacion'], var_name='date', value_name='tempsmin')
tempsmax = pd.melt(tempsmax, id_vars=['codigo_estacion'], var_name='date', value_name='tempsmax')

# These should be floats!
precipit['precipit'] = precipit['precipit'].astype('float32')
tempsavg['tempsavg'] = tempsavg['tempsavg'].astype('float32')
tempsmin['tempsmin'] = tempsmin['tempsmin'].astype('float32')
tempsmax['tempsmax'] = tempsmax['tempsmax'].astype('float32')

# Reduces memory!
stations['codigo_estacion'] = stations['codigo_estacion'].astype('category')
precipit['codigo_estacion'] = precipit['codigo_estacion'].astype('category')
tempsavg['codigo_estacion'] = tempsavg['codigo_estacion'].astype('category')
tempsmin['codigo_estacion'] = tempsmin['codigo_estacion'].astype('category')
tempsmax['codigo_estacion'] = tempsmax['codigo_estacion'].astype('category')

#To continue reducing memory, we will turn the dates into two columns with int8
def split_date(df):
    ndf = df['date'].str.split("-", n=1, expand = True)
    df['ano'] = ndf[0].astype('category')
    df['mes'] = ndf[1].astype('category')
    df.drop('date', inplace=True, axis=1)
    
split_date(precipit)
split_date(tempsavg)
split_date(tempsmin)
split_date(tempsmax)

stations.set_index(['codigo_estacion'], inplace=True)
precipit.set_index(['codigo_estacion', 'ano', 'mes'], inplace=True)
tempsavg.set_index(['codigo_estacion', 'ano', 'mes'], inplace=True)
tempsmin.set_index(['codigo_estacion', 'ano', 'mes'], inplace=True)
tempsmax.set_index(['codigo_estacion', 'ano', 'mes'], inplace=True)

print(' DONE')
print('stations:', stations.columns.values)
print('precipit:', precipit.columns.values)
print('tempsavg:', tempsavg.columns.values)
print('tempsmin:', tempsmin.columns.values)
print('tempsmax:', tempsmax.columns.values)

print('\nMerging everything...', end='')
data = pd.merge(
    stations,
    pd.merge(
        pd.merge(precipit, tempsavg, on=['codigo_estacion', 'ano', 'mes'], copy=False),
        pd.merge(tempsmin, tempsmax, on=['codigo_estacion', 'ano', 'mes'], copy=False),
        on=['codigo_estacion', 'ano', 'mes'], copy=False
    ).reset_index(level=[1,2]),
    on=['codigo_estacion'], copy=False
).reset_index()
print(' DONE')

print('\nReplacing -9999 with nan')
data.replace(-9999, np.nan, inplace=True)

display(data.head(10))

Loading CSVs...

FileNotFoundError: [Errno 2] File b'cr2_prAmon_2018_stations_ghcn.txt' does not exist: b'cr2_prAmon_2018_stations_ghcn.txt'

In [ ]:
data.to_csv('clima_chile.csv', index = None, header=True)
data.to_pickle('clima_chile.xz')